# Dataset Statistics for Perception Package Projects
This example notebook shows how to use datasetinsights to load synthetic datasets generated from the [Perception package](https://github.com/Unity-Technologies/com.unity.perception) and visualize dataset statistics. It includes statistics and visualizations of the outputs built into the Perception package and should give a good idea of how to use datasetinsights to visualize custom annotations and metrics.

## Setup dataset
If the dataset was generated locally, point `data_root` below to the path of the dataset. The `GUID` folder suffix should be changed accordingly.   

In [ ]:
data_root = "/data/<GUID>"

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the metrics needed for dataset statistics.

Provide the `run-execution-id` which generated the dataset and a valid `auth_token` in the following cell. The `auth-token` can be generated using the Unity Simulation [CLI](https://github.com/Unity-Technologies/Unity-Simulation-Docs/blob/master/doc/cli.md#usim-inspect-auth).

In [ ]:
# import os
# from datasetinsights.data.simulation.download import download_manifest, Downloader
# data_volume = "/data"   # directory where datasets should be downloaded to and loaded from
# run_execution_id = "ojEawoj"      # Unity Simulation run-execution-id
# auth_token = "xxxx"     # Unity Simulation auth token
# project_id = "xxxx"   # Unity Project ID

# data_root = os.path.join(data_volume, run_execution_id)

Before loading the dataset metadata for statistics we first download the relevant files from Unity Simulation. For downloading files, Unity Simulation provides a manifest file providing file paths and signed urls for each file. `download_manifest()` will download the manifest file to disk. `Download` can then be used to download the metrics and metric definitions.

In [ ]:
# manifest_file = os.path.join(data_volume, f"{run_execution_id}.csv")
# download_manifest(run_execution_id, manifest_file, auth_token, project_id)

# dl = Downloader(manifest_file, data_root, use_cache=True)
# dl.download_references()
# dl.download_metrics()

## Load dataset metadata
Once the dataset metadata is downloaded, it can be loaded for statistics using `datasetinsights.data.simulation`. Annotation and metric definitions are loaded into pandas dataframes using `AnnotationDefinitions` and `MetricDefinitions` respectively.

In [ ]:
import datasetinsights.data.simulation as sim
ann_def = sim.AnnotationDefinitions(data_root)
ann_def.table

In [ ]:
metric_def = sim.MetricDefinitions(data_root)
metric_def.table

## Built-in Statistics
The following tables and charts are supplied by `datasetinsights.data.datasets.statistics.RenderedObjectInfo` on datasets that include the "rendered object info" metric.

In [ ]:
import datasetinsights.data.datasets.statistics as stat
import datasetinsights.data.simulation.metrics as metrics
from datasetinsights.data.simulation.exceptions import DefinitionIDError
from datasetinsights.visualization.plots import bar_plot, histogram_plot, rotation_plot

max_samples = 10000          # maximum number of samples points used in histogram plots

rendered_object_info_definition_id = "5ba92024-b3b7-41a7-9d3f-c03a6a8ddd01"

roinfo = None
try:
    roinfo = stat.RenderedObjectInfo(data_root=data_root, def_id=rendered_object_info_definition_id)
except DefinitionIDError:
    print("No RenderedObjectInfo in this dataset")

### Descriptive Statistics

In [ ]:
if roinfo is not None:
    print(roinfo.num_captures())
    roinfo.raw_table.head(3)

### Total Object Count

In [ ]:
if roinfo is not None:
    total_count = roinfo.total_counts()
    display(total_count)
    
    display(bar_plot(
        total_count, 
        x="label_id", 
        y="count", 
        x_title="Label Name",
        y_title="Count",
        title="Total Object Count in Dataset",
        hover_name="label_name"
    ))

### Per Capture Object Count

In [ ]:
if roinfo is not None:
    per_capture_count = roinfo.per_capture_counts()
    display(per_capture_count.head(10))

In [ ]:
if roinfo is not None:
    display(histogram_plot(
        per_capture_count, 
        x="count",  
        x_title="Object Counts Per Capture",
        y_title="Frequency",
        title="Distribution of Object Counts Per Capture",
        max_samples=max_samples
    ))

### Object Visible Pixels

In [ ]:
if roinfo is not None:
    display(histogram_plot(
        roinfo.raw_table, 
        x="visible_pixels",  
        x_title="Visible Pixels Per Object",
        y_title="Frequency",
        title="Distribution of Visible Pixels Per Object",
        max_samples=max_samples
    ))

## Annotation Visualization
In the following sections we show how to load annotations from the Captures object and visualize them. Similar code can be used to consume annotations for model training or visualize and train on custom annotations.

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the images, captures and annotations in the dataset. Make sure you have enough disk space to store all files. For example, a dataset with 100K captures requires roughly 300GiB storage.

In [ ]:
# dl.download_captures()
# dl.download_binary_files()

### Load captures

In [ ]:
cap = sim.Captures(data_root)
cap.captures.head(3)

### Bounding Boxes
In this section we render 2d bounding boxes on top of the captured images.

In [ ]:
import os

from ipywidgets import interact, interactive, fixed, interact_manual
from PIL import Image
from datasetinsights.visualization.plots import plot_bboxes
from datasetinsights.data.datasets.synthetic import read_bounding_box_2d

bounding_box_definition_id = "f9f22e05-443f-4602-a422-ebe4ea9b55cb"
line_width = 5
def draw_bounding_boxes(index):
    filename = os.path.join(data_root, box_captures.loc[index, "filename"])
    annotations = box_captures.loc[index, "annotation.values"]
    image = Image.open(filename)
    boxes = read_bounding_box_2d(annotations)
    img_with_boxes = plot_bboxes(image, boxes, box_line_width=line_width)
    img_with_boxes.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_boxes)

try:
    box_captures = cap.filter(def_id=bounding_box_definition_id)
    interact(draw_bounding_boxes, index=(0, box_captures.shape[0]))
except DefinitionIDError:
    print("No bounding boxes found")

## Semantic Segmentation
In this section we render the semantic segmentation images on top of the captured images.

In [ ]:
def draw_with_segmentation(index, opacity):
    filename = os.path.join(data_root, seg_captures.loc[index, "filename"])
    seg_filename = os.path.join(data_root, seg_captures.loc[index, "annotation.filename"])
    
    image = Image.open(filename)
    seg = Image.open(seg_filename)
    img_with_seg = Image.blend(image, seg, opacity)
    img_with_seg.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_seg)
    
try:
    semantic_segmentation_definition_id = "12f94d8d-5425-4deb-9b21-5e53ad957d66"
    seg_captures = cap.filter(def_id=semantic_segmentation_definition_id)
    interact(draw_with_segmentation, index=(0, seg_captures.shape[0]), opacity=(0.0, 1.0))
except DefinitionIDError:
    print("No semantic segmentation images found")